[Documentação](https://python.langchain.com/docs/how_to/#retrievers)

#### 1. O que é um Retriever?

Um **Retriever** é um componente responsável por, dado um texto de consulta (query), **retornar uma lista de documentos relevantes**. O conceito de “documentos” pode variar – podem ser pedaços de texto, registros em um banco de dados, páginas da web etc. 

É importante salientar que um retriever é um `runnable`, ou seja, você pode invocá-lo. Como retorno teremos um `Document`. Cada documento é frequentemente representado por:

- **page_content**: o conteúdo do documento (string).
- **metadata**: dados adicionais (por exemplo, fonte, autor, ID, data).

A interface de retriever no LangChain é bastante simples:

1. **Entrada**: uma query (string)
2. **Saída**: lista de objetos `Document` relevantes para a query

Por trás desse processo, o retriever pode usar qualquer método de busca, desde pesquisas lexicais (BM25, por exemplo) até buscas semânticas em **bancos de dados vetoriais**. O ponto comum é que, para a aplicação final, basta chamar o retriever com a query, e ele entrega os documentos relevantes. Esse é o nível de abstração que o LangChain te entrega.

#### 2. Usando um Vector Store como Retriever

Hoje em dia, um dos métodos mais populares (e eficientes) de realizar buscas relevantes em textos não-estruturados é por meio de **bancos de dados vetoriais (vector stores)**. A LangChain permite converter esses bancos em retrievers de forma direta.

#### 2.1 Criação de um Vector Store Retriever

Suponha que já tenhamos um vector store pronto (por exemplo, usando Chroma, Qdrant, Milvus, Pinecone etc.). Para transformar esse vector store em um retriever, basta chamar o método `as_retriever()`:

```python
vectorstore = MyVectorStore()  # Ex.: Qdrant, Chroma, Pinecone...
retriever = vectorstore.as_retriever()
```

Com esse `retriever`, podemos chamá-lo de forma padronizada, por exemplo usando `invoke` (ou `.get_relevant_documents(query)`, dependendo da versão da LangChain):

```python
docs = retriever.invoke("Qual foi a declaração do aluno sobre IA?")
for doc in docs:
    print(doc.page_content)
```

#### 2.2 Personalizando o Tipo de Busca

No caso de `vector stores` (banco de dados vetorial), a busca pode variar, ou seja, você pode pre configurar como deve ser a estratégia de similaridade parametrizando o parâmetro `search_type` e `search_kwargs`:

- **similarity_search** (padrão): busca documentos com maior similaridade vetorial.
- **maximum marginal relevance (mmr)**: procura variar mais os resultados (diversificar). Por exemplo, `search_type="mmr"`
- **limite mínimo de score** (threshold): só retorna documentos que atinjam um nível mínimo de similaridade. Por exemplo, `search_type="similarity_score_threshold"`
- **top-k**: permite definir quantos documentos retornar. Por exemplo: `search_kwargs={"k": 1}`


In [1]:
%run ../helpers/00-llm.ipynb

In [2]:
from helpers.llm import initialize_llm, logger, pretty_print_docs
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader , PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.retrievers.multi_query import MultiQueryRetriever

llm, _, embeddings = initialize_llm()

NOME_COLECAO="FAQ_BOOKING_COM"

INFO:helpers.llm:Using AzureOpenAI.


In [9]:
lista_documentos = PyPDFLoader('data/FAQ_BOOKING_COM.pdf').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documentos = text_splitter.split_documents(lista_documentos) 
pretty_print_docs(documentos)

Document 1:

Perguntas Frequentes Booking.com 
Pagamentos: 
 
Recebi uma cobrança indevida. O que devo fazer? Caso você tenha recebido uma 
cobrança indevida da Booking.com: por favor, visite a Central de Ajuda. É possível que a 
gente precise de uma cópia do seu extrato bancário mostrando a cobrança indevida, além 
do número de confirmação e o código PIN da sua reserva. 
Caso você tenha recebido uma cobrança indevida da acomodação: por favor, entre em 
contato diretamente com eles. 
 
Onde posso encontrar a confirmação de que paguei por minha reserva? Você encontrará 
a confirmação de pagamento no seu e-mail de confirmação. No e-mail, também existe a 
opção de fazer o download da confirmação de pagamento em PDF . 
 
Onde posso ver a política de pagamento da minha reserva? A Booking.com irá cobrar o 
seu cartão pelo preço total da reserva quando ela for feita. Caso esteja procurando por 
mais informações sobre seu pagamento, por favor, verifique seu e-mail de confirmação.
-------------

In [10]:
# Cria o banco de dados vetorial, gerando os embeddings dos documentos
Chroma.from_documents(documentos, collection_name=NOME_COLECAO, embedding=embeddings, persist_directory="./FAQ_BOOKING_COM")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:httpx:HTTP Request: POST https://trainning-grassato-01.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [11]:
# Conecta-se ao banco vetorial já existente
db = Chroma(  
    persist_directory="./FAQ_BOOKING_COM",  
    collection_name=NOME_COLECAO,  
    embedding_function=embeddings,  
)

In [13]:
# Banco Vetorial como Recuperador:
db_retriever = db.as_retriever()

# Banco Vetorial como Recuperador, configurando parâmetros:
#db_retriever = db.as_retriever(search_kwargs={'k': 5, })
#db_retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 10})


In [ ]:
# Exemplo de consulta no banco:
query = "Cancelaram minha reserva o que fazer?"
# query = "Quando eu chego na hospedagem preciso pagar algo?"
pedacos_retornados = db_retriever.invoke(query)


print(f"Total de pedaços retornados (documents): {len(pedacos_retornados)}")

for i, pedaco in enumerate(pedacos_retornados):
    print(f"------ (documents) chunk {i} -------")
    print(pedaco.page_content)
    print("-------------------------------------")

In [14]:
# Banco Vetorial como Recuperador:
# db_retriever = db.as_retriever()

# Banco Vetorial como Recuperador, configurando parâmetros:
db_retriever = db.as_retriever(search_kwargs={'k': 5, })
#db_retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 10})

retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db_retriever, llm=llm)

In [15]:
# Exemplo de consulta no banco:
# query = "Cancelaram minha reserva o que fazer?"
query = "Quando eu chego na hospedagem preciso pagar algo?"

# Executando o recuperador:
pedacos_retornados = db_retriever.invoke(query)


print(f"Total de pedaços retornados (documents): {len(pedacos_retornados)}\n")

for i, pedaco in enumerate(pedacos_retornados):
    print(f"------ (documents) chunk {i} -------")
    print(pedaco.page_content)
    print("-------------------------------------")


INFO:httpx:HTTP Request: POST https://trainning-grassato-01.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Total de pedaços retornados (documents): 5

------ (documents) chunk 0 -------
seu cartão pelo preço total da reserva quando ela for feita. Caso esteja procurando por 
mais informações sobre seu pagamento, por favor, verifique seu e-mail de confirmação. 
 
Houve a cobrança. Preciso fazer alguma coisa? Não, você não precisa fazer nada. 
Fizemos a cobrança do preço da reserva. Se houver algum imposto ou taxa adicional, 
essas informações devem constar na página da propriedade. 
 
Vou precisar pagar algum valor adicional na chegada? Às vezes é necessário pagar 
impostos e taxas na chegada. Se esse for o caso, o valor é indicado antes da reserva e no 
seu e-mail de confirmação. Se não houver a indicação de nenhum valor, você não precisa 
pagar nada na chegada. 
Se a acomodação fizer alguma cobrança e você achar que ela é indevida, por favor, visite 
a Central de Ajuda. 
 
Como eu incluo a minha reserva na minha conta? Infelizmente, não é possível incluir essa 
reserva na sua conta. Estamos

O LLM produz variações da pergunta, por exemplo:

- “Como podemos dividir problemas em etapas menores?”
- “Quais técnicas de decomposição de tarefas são mais utilizadas?”
- etc.

Depois, o multi-query faz a busca para cada variação e combina os documentos relevantes, resultando em uma visão **mais abrangente** do assunto.

Leia mais em: https://python.langchain.com/docs/how_to/MultiQueryRetriever/

### 4.3 Outros

Existem outros tipos de retrievers que valem a pena consultar na fonte da documentação:

- [Self Query](https://python.langchain.com/docs/how_to/self_query/)
- [Contextual Compression](https://python.langchain.com/docs/how_to/contextual_compression/)

## 5. Conclusão

### 5.1 Por que usar Retrievers?

Os **retrievers** desempenham um papel fundamental em aplicações de **IA e NLP** que precisam obter informações relevantes de grandes volumes de dados ou misturar diferentes fontes de conhecimento. A interface padronizada da LangChain:

- Simplifica o processo de alternar ou combinar diferentes métodos de busca.
- Fornece um ponto único de integração com LLMs para criar **aplicações que buscam e respondem** a perguntas com mais precisão e contexto.

### 5.2 Recomendações Finais

- Avalie o tipo de dado e o caso de uso para escolher o método de busca.
- Se os resultados estiverem muito homogêneos ou superficiais, considere **modelos avançados** (como MMR) ou o uso de **MultiQueryRetriever**.

Com esses conceitos, você terá uma base sólida para adicionar **retrieval** (recuperação de informação) em suas aplicações, integrando com LLMs ou outras ferramentas de NLP para obter respostas relevantes e contextualizadas.